In [2]:
!pip install -r requirements.txt
import requests
import time
from bs4 import BeautifulSoup


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [36]:
base_url = 'https://visitseattle.org/events/page/{}'
all_events = []

for page_number in range(1, 39):  # Iterate over pages 1 to 38
    url = base_url.format(page_number)
    res = requests.get(url)
    
    if res.status_code == 200:
        soup = BeautifulSoup(res.text, 'html.parser')
        selector = "div.search-result-preview > div > h3 > a"
        a_eles = soup.select(selector)
        
        # Extract href attributes from the anchor elements and append to the list
        all_events.extend([x['href'] for x in a_eles])

# Print the list of event URLs
print("event summary:", len(all_events))
print(all_events)

event summary: 341
['https://visitseattle.org/events/dead-bars/', 'https://visitseattle.org/events/emotional-a-night-for-the-heartbroken/', 'https://visitseattle.org/events/loonieboys/', 'https://visitseattle.org/events/corteo/', 'https://visitseattle.org/events/bewitcher/', 'https://visitseattle.org/events/burn-burn-burn/', 'https://visitseattle.org/events/carly-ann-calbero/', 'https://visitseattle.org/events/el-primer-instinto/', 'https://visitseattle.org/events/gary-janetti/', 'https://visitseattle.org/events/jammah/', 'https://visitseattle.org/events/saturday-family-concert-the-harmonica-pocket/', 'https://visitseattle.org/events/seattle-thunderbirds-vs-tri-city-americans-2/', 'https://visitseattle.org/events/hokusai-inspiration-and-influence-from-the-collection-of-the-museum-of-fine-arts-boston/', 'https://visitseattle.org/events/an-evening-with-chris-botti/', 'https://visitseattle.org/events/blueys-big-play/', 'https://visitseattle.org/events/mike-birbiglia-please-stop-the-ride/'

In [38]:
def extract_event_details(detail_url):
    detail_res = requests.get(detail_url)

    if detail_res.status_code == 200:
        detail_soup = BeautifulSoup(detail_res.text, 'html.parser')
        # Extracting details (modify selectors as needed)
        name = detail_soup.select_one('div.medium-6.columns.event-top > h1').text.strip()
        date_time = detail_soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(1)').text.strip()
        location = detail_soup.select_one('div.medium-6.columns.event-top > h4 > span:nth-child(2)').text.strip()
        event_type = detail_soup.select_one('div.medium-6.columns.event-top > a:nth-child(3)').text.strip()
        region = detail_soup.select_one('div.medium-6.columns.event-top > a:nth-child(4)').text.strip()
        
        return [name, date_time, location, event_type, region]
    else:
        return None

In [39]:
all_event_details = []
for event_url in all_events:
    event_details = extract_event_details(event_url)
    
    if event_details:
        all_event_details.append(event_details)

In [40]:
import csv
csv_header = ['Name', 'Date + Time', 'Location', 'Type', 'Region']
with open('events.csv', 'w', newline='', encoding='utf-8') as csvfile:
    csv_writer = csv.writer(csvfile)
    
    # Write header
    csv_writer.writerow(csv_header)
    
    # Write event details
    csv_writer.writerows(all_event_details)

print("Event details have been stored in events.csv")

Event details have been stored in events.csv


# Weather

In [3]:
url = "https://api.weather.gov/points/39.7456,-97.0892"
res = requests.get(url)
res

<Response [200]>

In [4]:
res.text

'{\n    "@context": [\n        "https://geojson.org/geojson-ld/geojson-context.jsonld",\n        {\n            "@version": "1.1",\n            "wx": "https://api.weather.gov/ontology#",\n            "s": "https://schema.org/",\n            "geo": "http://www.opengis.net/ont/geosparql#",\n            "unit": "http://codes.wmo.int/common/unit/",\n            "@vocab": "https://api.weather.gov/ontology#",\n            "geometry": {\n                "@id": "s:GeoCoordinates",\n                "@type": "geo:wktLiteral"\n            },\n            "city": "s:addressLocality",\n            "state": "s:addressRegion",\n            "distance": {\n                "@id": "s:Distance",\n                "@type": "s:QuantitativeValue"\n            },\n            "bearing": {\n                "@type": "s:QuantitativeValue"\n            },\n            "value": {\n                "@id": "s:value"\n            },\n            "unitCode": {\n                "@id": "s:unitCode",\n                "@typ

In [5]:
point_dict = res.json()
point_dict

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   's': 'https://schema.org/',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#',
   'geometry': {'@id': 's:GeoCoordinates', '@type': 'geo:wktLiteral'},
   'city': 's:addressLocality',
   'state': 's:addressRegion',
   'distance': {'@id': 's:Distance', '@type': 's:QuantitativeValue'},
   'bearing': {'@type': 's:QuantitativeValue'},
   'value': {'@id': 's:value'},
   'unitCode': {'@id': 's:unitCode', '@type': '@id'},
   'forecastOffice': {'@type': '@id'},
   'forecastGridData': {'@type': '@id'},
   'publicZone': {'@type': '@id'},
   'county': {'@type': '@id'}}],
 'id': 'https://api.weather.gov/points/39.7456,-97.0892',
 'type': 'Feature',
 'geometry': {'type': 'Point', 'coordinates': [-97.0892, 39.7456]},
 'properties': {'@id': 'https://api.weather.gov/p

In [6]:
point_dict.keys()

dict_keys(['@context', 'id', 'type', 'geometry', 'properties'])

In [7]:
forcast_url = point_dict['properties']['forecast']
forcast_url

'https://api.weather.gov/gridpoints/TOP/32,81/forecast'

In [8]:
res = requests.get(forcast_url)
res.json()

{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
  {'@version': '1.1',
   'wx': 'https://api.weather.gov/ontology#',
   'geo': 'http://www.opengis.net/ont/geosparql#',
   'unit': 'http://codes.wmo.int/common/unit/',
   '@vocab': 'https://api.weather.gov/ontology#'}],
 'type': 'Feature',
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-97.1089731, 39.7668263],
    [-97.1085269, 39.7447788],
    [-97.0798467, 39.7451195],
    [-97.08028680000001, 39.767167],
    [-97.1089731, 39.7668263]]]},
 'properties': {'updated': '2024-01-19T23:26:27+00:00',
  'units': 'us',
  'forecastGenerator': 'BaselineForecastGenerator',
  'generatedAt': '2024-01-19T23:34:06+00:00',
  'updateTime': '2024-01-19T23:26:27+00:00',
  'validTimes': '2024-01-19T17:00:00+00:00/P7DT8H',
  'elevation': {'unitCode': 'wmoUnit:m', 'value': 441.96},
  'periods': [{'number': 1,
    'name': 'This Afternoon',
    'startTime': '2024-01-19T17:00:00-06:00',
    'endTime': '2024-01-19T18:00:00-06:00',
   

# Openstreet map API